In [2]:
import requests

API_KEY = "1d48b5e24b27cd111582c21dcff9b8f5"
BASE_URL = "https://api.themoviedb.org/3"

def get_json(url, params=None):
    """A function to make get requests and return a json file"""

    params = params or {}
    params['api_key'] = API_KEY

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Lève une erreur si erreur API
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur API : {e}")
        return None 

def get_tmdb_id(imdb_id):
    """A funtion to get TMBD id from IMDb id"""

    json_file = get_json(f"{BASE_URL}/find/{imdb_id}", {"external_source" : "imdb_id"})

    if json_file == None : # Return None if API Error
        return None
    elif len(json_file['movie_results']) == 0 : # Return 0 if not in TMDB database
        return 0
    else :
        return json_file['movie_results'][0]['id']

def get_movie_details(movie_id):
    """A function to get detailed movie info by movie ID"""

    details = get_json(f"{BASE_URL}/movie/{movie_id}", {"append_to_response": "credits"})

    if details != None : 

        return {
            "tmdb_id" : movie_id,
            "imdb_id" : details.get("imdb_id"),
            "original_language" : details.get("original_language"),
            "popularity" : details.get("popularity"),
            "overview" : details.get("overview"), 
            "budget" : details.get("budget"),
            "country": details.get("origin_country"),
            "production companies" : [c["name"] for c in details.get("production_companies", [])],
            "revenue" : details.get("revenue"),
            "vote_average" : details.get("vote_average"),
            "vote_count" : details.get("vote_count"),
            "cast": [(c["name"],c["gender"],c["popularity"]) for c in details["credits"].get("cast",[])[:10]]  # main 10 actors
        }
    else : # If API Error
        return None 

In [ ]:
import pandas as pd

def fetch_full_movie_data(imdb_id):
    """Fonction maître"""
    
    tmdb_id = get_tmdb_id(imdb_id)
    if tmdb_id:
        details = get_movie_details(tmdb_id)
        return details
            
    print(f"Échec de la récupération pour {imdb_id}")
    return None

imdb_id_list = ["tt0133093", "tt0137523", "tt0068646", "tt_ID_FOIREUX"] 


# Liste des résultats
results_list = []
fails = []

# Ajout de tous les films
for imdb_id in imdb_id_list:
    data = fetch_full_movie_data(imdb_id)
    if data:  # On n'ajoute que les succès
        results_list.append(data)
    else:
        fails.append(imdb_id)

# Création du dataframe
df = pd.DataFrame(results_list)

df.head()

Échec de la récupération pour tt_ID_FOIREUX


,tmdb_id,imdb_id,original_language,popularity,overview,budget,country,production companies,revenue,vote_average,vote_count,cast
0,603,tt0133093,en,20.0929,"Set in the 22nd century, The Matrix tells the ...",63000000,[US],"[Village Roadshow Pictures, Groucho II Film Pa...",463517383,8.234,26959,"[(Keanu Reeves, 2, 8.7176), (Laurence Fishburn..."
1,550,tt0137523,en,16.5831,A ticking-time-bomb insomniac and a slippery s...,63000000,[US],"[Fox 2000 Pictures, Regency Enterprises, Linso...",100853753,8.438,30945,"[(Edward Norton, 2, 5.8281), (Brad Pitt, 2, 7...."
2,238,tt0068646,en,31.8334,"Spanning the years 1945 to 1955, a chronicle o...",6000000,[US],"[Paramount Pictures, Alfran Productions]",245066411,8.685,22020,"[(Marlon Brando, 2, 1.8059), (Al Pacino, 2, 3...."


In [4]:
url_TMDB = 'https://api.themoviedb.org/3/find/tt1375666?external_source=imdb_id'
get_movie_details(get_tmdb_id('tt1375666'))

{'tmdb_id': 27205,
 'imdb_id': 'tt1375666',
 'original_language': 'en',
 'popularity': 20.9485,
 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.',
 'budget': 160000000,
 'country': ['US', 'GB'],
 'production companies': ['Legendary Pictures',
  'Syncopy',
  'Warner Bros. Pictures'],
 'revenue': 839030630,
 'vote_average': 8.37,
 'vote_count': 38168,
 'cast': [('Leonardo DiCaprio', 2, 5.9459),
  ('Joseph Gordon-Levitt', 2, 3.6509),
  ('Ken Watanabe', 2, 3.8215),
  ('Tom Hardy', 2, 6.0105),
  ('Elliot Page', 3, 4.4884),
  ('Dileep Rao', 2, 0.9523),
  ('Cillian Murphy', 2, 4.7981),
  ('Tom Berenger', 2, 2.3135),
  ('Marion Cotillard', 1, 3.8582),
  ('Pete Postlethwaite', 2, 1.5138)]}